<a href="https://colab.research.google.com/github/swoo-nam/project_final_team1/blob/main/%EC%9D%B4%EC%86%8C%ED%98%84_%EC%84%9C%EB%B9%84%EC%8A%A42_%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A0%95%EC%A0%9C5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os
import re

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/Final_Project/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

# 요기요 데이터

In [ ]:
yogiyo = pd.read_csv(f"{DATA_PATH}yogiyo_reviews_all.csv")
yogiyo

,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,5,5,5,5.0,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,5,5,5,5.0,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,5,5,5,5.0,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...
3,굿 말이 필요 없는 메뉴 였습니다,5,5,5,5.0,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...
4,맛있어요 양도많구좋음,5,5,5,5.0,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...
...,...,...,...,...,...,...
18681,맛있게 잘 먹었어요^^,5,5,5,5.0,감사합니다!!! 다음에도 이용해주세용 ㅎㅎ
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요… 그래도 잘 먹었습니다,4,5,5,5.0,ㅠㅠ 죄송합니다 !! 다음에도 애용해주세용 ㅎㅎ
18683,배달 빨라요 맛있게 잘 먹었습니당 !,5,5,5,5.0,감사합니다!!! 다음에도 이용해주세용 ㅎㅎ
18684,잘 먹었어요~~~~~,5,5,5,5.0,감사합니다!!! 다음에도 이용해주세용 ㅎㅎ


In [ ]:
for i in range(1, 6):
    num_rating = yogiyo[yogiyo['별점'] == i].shape[0]
    print(f"별점{i}점 개수: {num_rating}, {num_rating / yogiyo.shape[0] * 100}%")

별점1점 개수: 92, 0.49234721181633306%
별점2점 개수: 64, 0.3425024082200578%
별점3점 개수: 179, 0.9579364229904741%
별점4점 개수: 560, 2.9968960719255056%
별점5점 개수: 17791, 95.21031788504763%


In [ ]:
for i in range(1, 6):
    num_rating = yogiyo[yogiyo['맛별점'] == i].shape[0]
    print(f"맛별점{i}점 개수: {num_rating}, {num_rating / yogiyo.shape[0] * 100}%")

맛별점1점 개수: 92, 0.49234721181633306%
맛별점2점 개수: 74, 0.3960184095044418%
맛별점3점 개수: 228, 1.220164829283956%
맛별점4점 개수: 687, 3.676549288237183%
맛별점5점 개수: 17605, 94.21492026115808%


In [ ]:
for i in range(1, 6):
    num_rating = yogiyo[yogiyo['양별점'] == i].shape[0]
    print(f"양별점{i}점 개수: {num_rating}, {num_rating / yogiyo.shape[0] * 100}%")

양별점1점 개수: 65, 0.3478540083484962%
양별점2점 개수: 57, 0.305041207320989%
양별점3점 개수: 200, 1.0703200256876806%
양별점4점 개수: 773, 4.136786899282885%
양별점5점 개수: 17591, 94.13999785935995%


In [ ]:
for i in range(1, 6):
    num_rating = yogiyo[yogiyo['배달별점'] == i].shape[0]
    print(f"배달별점{i}점 개수: {num_rating}, {num_rating / yogiyo.shape[0] * 100}%")

배달별점1점 개수: 155, 0.8294980199079525%
배달별점2점 개수: 70, 0.3746120089906882%
배달별점3점 개수: 198, 1.0596168254308038%
배달별점4점 개수: 523, 2.7988868671732847%
배달별점5점 개수: 16805, 89.93364015840737%


In [ ]:
yogiyo.loc[(yogiyo.양별점 == 1)]

,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글
96,리뷰 이벤트 한다지만 너무 아닌 거 같아서 적습니다. 배달이 20분 넘게 안오길래 ...,1,1,1,1.0,안녕하세요 1988응답하라추억의옛날도시락 입니다.\n\n주신 리뷰 잘 읽었습니다. ...
144,진짜리뷰안쓰는데 양이 2만원씩내고 먹을양은 절대아닙니다 \n장난하는것도아니고,1,2,1,5.0,솔직한 리뷰 남겨 주셔서 감사합니다. \n\n저희 1988응답하라추억의옛날도시락은 ...
252,만들어 놓은지 한참 된 커피 ….배달 예정 시간 25분이 지났어요…….,1,1,1,1.0,안녕하세요 배달이 늦어져 죄송합니다. 한파로 길이 얼어서 배달기사님들이 상황이 여유...
658,공용 비번이있어서 저녁이라 바쁘실꺼 그냥 말 없이 기다렸는데 평가 할 수 없이 다 ...,1,1,1,1.0,안녕하세요 가게 대표입니다.\n내용 전달받았고 통화녹음내용까지 모두 확인했습니다.\...
1552,음.....양도 적고 간도 짜고....잘못시켰어요...그냥 더 이야기는 안할게요 설...,1,1,1,1.0,안녕하세요\n매장이용에 불편을 드려 죄송합니다\n항상 고객님들이 기분좋게 식사하실 ...
...,...,...,...,...,...,...
15911,맛은 평범합니다. 영수증에 주소까지나와있는데 전화하셔서 제대로 안나와있다고하셔서 사...,1,2,1,1.0,기분나쁘셨다니 죄송합니다.\n기사님이 뭔가 착오가 있으셨나봅니다. 제가 오늘 대책을...
16088,자장면은 멸치맛이 올라오고 짬뽕은 해물도 별루없고 탕수육은 눅눅하고 거의 대부분 버...,1,1,1,5.0,짜장면에 멸치가안들어가는데 어떻게멸지맛이나는지 모르겠네요 ㅜㅜ입에안맞으신다니 안타갑...
16401,하진짜 사진도 아깝네요^^ 면이 젓가락으로 집으면 그냥으깨져서 먹을수가없네요 콜라는...,1,1,1,1.0,안녕하세요 \n우선 사과말씀부터드립니다\n정말죄송합니다\n오늘비가많이오다보니 기사님...
17679,음료 맛도 별로였고 빨대도 안주고 마감전이라 그런지 샌드위치 시켰는데 먹다가 그냥...,1,1,1,1.0,고객님 ㅜㅜ\n먼저 죄송하단 말씀 드립니다 …\n어제 저희 직원이 실수로 빨대를 \...


## label을 임의로 만들어보자

- 별점 1,2,3 -> 0 / 4,5 -> 1
- 맛별점, 양별점, 배달별점에 적용하여 새로운 column 생성
- 경우의 수를 조합하여 총 8개의 label 생성

In [ ]:
yogiyo.loc[(yogiyo.맛별점 == 1)|(yogiyo.맛별점 == 2)|(yogiyo.맛별점 == 3)]

,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,amount,taste,delivery
43,맛이나 양은 무난해요\n그런데...\n배달예정시간이 무색하게 늦게도착ㅜㅜ\n차라리 ...,3,3,4,1.0,1988응답하라추억의옛날도시락을 선택해 주셔서 감사합니다. \n\n저희 매장에서는 ...,1,0,0
47,"메인은 좋았는데,\n아... 반찬이 다 쉬어 모두 버렸어요\n한번 씹는데 촉감,맛...",3,2,3,2.0,안녕하세요 1988응답하라추억의옛날도시락 입니다.\n\n우선 감사하게도 저희 매장을...,0,0,0
51,예정시간보다 30분이 지나서야 도착하고 음식도 다 식어서 왔네요,3,3,4,1.0,1988응답하라추억의옛날도시락을 선택해 주셔서 감사합니다. \n\n저희 매장에서는 ...,1,0,0
52,배달이 넘 느려서 우선 감점 배가 고픈데 그럼에도 맛은 별로 개취존중~스팸이 젤 맛...,2,2,5,2.0,1988응답하라추억의옛날도시락을 선택해 주셔서 감사합니다. \n\n저희 매장에서는 ...,1,0,0
57,배달 늦게옴 \n전화주면 받으러나간다니깐 전화 안줌,2,3,3,1.0,저희 1988응답하라추억의옛날도시락을 찾아 주셔서 감사합니다. \n\n맛있게 드시는...,0,0,0
...,...,...,...,...,...,...,...,...,...
18244,매워서 못먹겠어요 코코낫 음료 넣었는데도 짜증나내요\n전화도 안받고,2,3,5,5.0,죄송합니다ㅜ 점심시간 바쁜시간대에 전화가와서 받았는데 끊어져서 다시 전화못드렸습니다...,1,0,0
18422,맛있게 잘 먹었습니다\n제가 따로 포장해달라 메세지 적었는데 왜 같이 포장해주신건가...,3,3,3,5.0,안녕하세요^^\n맛있게 드셨다니 정말 기쁘네요😁😁\n근데 따로 포장 요청하셨는데\n...,1,0,0
18437,소고기인데 조개랑 섞여 있네요.,2,1,5,5.0,정보란에 보시면 모든 미역국에 국물의 진한 맛을 위해 조개가 들어간다고 공지를 하였...,1,0,0
18450,오복미역같이 진하지는 않으나 먹을만합니다\n배달비가 너무비싼건 아쉽네요..,3,3,3,2.0,이용해주셔서 감사합니다,0,0,0


In [ ]:
yogiyo['amount'] = 0

In [ ]:
yogiyo.loc[yogiyo['양별점'].isin([4,5]), 'amount'] = 1

In [ ]:
yogiyo['taste'] = 0

In [ ]:
yogiyo.loc[yogiyo['맛별점'].isin([4,5]), 'taste'] = 1

In [ ]:
yogiyo['delivery'] = 0

In [ ]:
yogiyo.loc[yogiyo['배달별점'].isin([4,5]), 'delivery'] = 1

In [ ]:
def set_label(yogiyo):
    if yogiyo['taste'] == 0 and yogiyo['amount'] == 0 and yogiyo['delivery'] == 0:
        return 0
    elif yogiyo['taste'] == 0 and yogiyo['amount'] == 0 and yogiyo['delivery'] == 1:
        return 1
    elif yogiyo['taste'] == 0 and yogiyo['amount'] == 1 and yogiyo['delivery'] == 0:
        return 2
    elif yogiyo['taste'] == 1 and yogiyo['amount'] == 0 and yogiyo['delivery'] == 0:
        return 3
    elif yogiyo['taste'] == 1 and yogiyo['amount'] == 1 and yogiyo['delivery'] == 0:
        return 4
    elif yogiyo['taste'] == 1 and yogiyo['amount'] == 0 and yogiyo['delivery'] == 1:
        return 5
    elif yogiyo['taste'] == 0 and yogiyo['amount'] == 1 and yogiyo['delivery'] == 1:
        return 6
    else:
        return 7

In [ ]:
yogiyo['label'] = yogiyo.apply(set_label, axis=1)

In [ ]:
for i in range(0, 8):
    num_rating = yogiyo[yogiyo['label'] == i].shape[0]
    print(f"label {i}점 개수: {num_rating}, {num_rating / yogiyo.shape[0] * 100}%")

label 0점 개수: 124, 0.6635984159263619%
label 1점 개수: 66, 0.3532056084769346%
label 2점 개수: 49, 0.26222840629348176%
label 3점 개수: 30, 0.1605480038531521%
label 4점 개수: 1155, 6.181098148346355%
label 5점 개수: 102, 0.5458632131007171%
label 6점 개수: 155, 0.8294980199079525%
label 7점 개수: 17005, 91.00396018409505%


In [ ]:
yogiyo.loc[(yogiyo.taste == 1)]

,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,amount,taste,delivery,label
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,5,5,5,5.0,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ...",1,1,1,7
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,5,5,5,5.0,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...,1,1,1,7
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,5,5,5,5.0,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...,1,1,1,7
3,굿 말이 필요 없는 메뉴 였습니다,5,5,5,5.0,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...,1,1,1,7
4,맛있어요 양도많구좋음,5,5,5,5.0,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...,1,1,1,7
...,...,...,...,...,...,...,...,...,...,...
18681,맛있게 잘 먹었어요^^,5,5,5,5.0,감사합니다!!! 다음에도 이용해주세용 ㅎㅎ,1,1,1,7
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요… 그래도 잘 먹었습니다,4,5,5,5.0,ㅠㅠ 죄송합니다 !! 다음에도 애용해주세용 ㅎㅎ,1,1,1,7
18683,배달 빨라요 맛있게 잘 먹었습니당 !,5,5,5,5.0,감사합니다!!! 다음에도 이용해주세용 ㅎㅎ,1,1,1,7
18684,잘 먹었어요~~~~~,5,5,5,5.0,감사합니다!!! 다음에도 이용해주세용 ㅎㅎ,1,1,1,7


## 특수문자 제거/맞춤법 및 띄어쓰기/ 가게 이름 추가해 놓은 csv 파일 불러오기

In [ ]:
modi = pd.read_csv(f"{DATA_PATH}요기요_가게_맞춤법수정.csv")
modi

,review,reply,store,corrected_reply
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최...",NaN,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛뿐만 아니라 양도, 서비스도 모두 최상..."
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,NaN,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,NaN,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...,NaN,수많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거 ...
4,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,NaN,수많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데 ...
...,...,...,...,...
18681,맛있게 잘 먹었어요,감사합니다! 다음에도 이용해주세용,NaN,감사합니다! 다음에도 이용해 주세요
18682,요청사항을 항상 잘 들어주셨는데 오늘은 못 모셨나봐요 그래도 잘 먹었습니다,죄송합니다 ! 다음에도 애용해주세용,NaN,죄송합니다! 다음에도 애용해 주세요
18683,배달 빨라요 맛있게 잘 먹었습니당 !,감사합니다! 다음에도 이용해주세용,NaN,감사합니다! 다음에도 이용해 주세요
18684,잘 먹었어요~,감사합니다! 다음에도 이용해주세용,NaN,감사합니다! 다음에도 이용해 주세요


In [ ]:
tmp = pd.concat([yogiyo,modi], axis=1)
tmp.head()

,고객리뷰,별점,맛별점,양별점,배달별점,사장댓글,amount,taste,delivery,label,review,reply,store,corrected_reply
0,사진보다 실물이 백배 맛있어요~~~~~~ 고기 양도 많아요~~~~~~ 배달시간도 딱...,5,5,5,5.0,"이렇게 저희 매장 찾아주셔서 감사드립니다!!\n\n맛 뿐만 아니라 양도, 서비스도 ...",1,1,1,7,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛 뿐만 아니라 양도, 서비스도 모두 최...",NaN,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛뿐만 아니라 양도, 서비스도 모두 최상..."
1,리뷰가 늦었네요.. 맛있게 잘 먹었습니다. 또 시킬게요!!,5,5,5,5.0,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요😇\n\n고객님께서 맛있다고 말씀 주시...,1,1,1,7,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,NaN,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,5,5,5,5.0,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. \n\n저희를 찾아 주신...,1,1,1,7,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,NaN,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...
3,굿 말이 필요 없는 메뉴 였습니다,5,5,5,5.0,수 많은 매장들을 헤치고 와주셔서 감사합니다!!\n\n저희 매장의 메뉴를 제대로 즐...,1,1,1,7,굿 말이 필요 없는 메뉴 였습니다,수 많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거...,NaN,수많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거 ...
4,맛있어요 양도많구좋음,5,5,5,5.0,수 많은 매장들을 헤치고 와주셔서 감사합니다😂\n\n메뉴 그 자체로 행복을 드리고 ...,1,1,1,7,맛있어요 양도많구좋음,수 많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데...,NaN,수많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데 ...


In [ ]:
tmp = tmp[['review','corrected_reply','store','label']]
tmp.head()

,review,corrected_reply,store,label
0,사진보다 실물이 백배 맛있어요~ 고기 양도 많아요~ 배달시간도 딱 맞춰서 보내주셨어요~,"이렇게 저희 매장 찾아주셔서 감사드립니다! 맛뿐만 아니라 양도, 서비스도 모두 최상...",NaN,7
1,리뷰가 늦었네요. 맛있게 잘 먹었습니다. 또 시킬게요!,좋은 리뷰 덕분에 기분이 행복해지는 것 같아요 고객님께서 맛있다고 말씀 주시니 피로...,NaN,7
2,잘 먹었습니다. 다만 주문에 있던 펩시콜라는 받지 못했습니다.,소중한 식사 시간에 저희 매장을 찾아 주셔서 감사드립니다. 저희를 찾아 주신 만큼 ...,NaN,7
3,굿 말이 필요 없는 메뉴 였습니다,수많은 매장들을 헤치고 와주셔서 감사합니다! 저희 매장의 메뉴를 제대로 즐기신 거 ...,NaN,7
4,맛있어요 양도많구좋음,수많은 매장들을 헤치고 와주셔서 감사합니다 메뉴 그 자체로 행복을 드리고 싶었는데 ...,NaN,7


## 오버샘플링(label 값에 따라서)

In [ ]:
from sklearn.utils import resample

# 오버샘플링을 수행할 별점 분포 계산
star_distribution = tmp['label'].value_counts().sort_index()

print(star_distribution)

0      124
1       66
2       49
3       30
4     1155
5      102
6      155
7    17005
Name: label, dtype: int64


In [ ]:
# 오버샘플링을 수행할 최대 샘플 수 설정
max_samples = 1000

# 오버샘플링 수행
oversampled_dfs = []

for label, count in star_distribution.items():
    subset = tmp[tmp['label'] == label]
    # 라벨이이 7이 아닌 경우에만 오버샘플링 수행
    if label == 7:
        oversampled_dfs.append(subset)
    elif label == 4:
        oversampled_dfs.append(subset)
    else:
        resampled = resample(subset, replace=True, n_samples=max_samples, random_state=SEED)
        oversampled_dfs.append(resampled)

In [ ]:
OV_train = pd.concat(oversampled_dfs).reset_index(drop=True)

In [ ]:
# 오버샘플링 후 별점 분포 확인
OV_train['label'].value_counts().sort_index()

0     1000
1     1000
2     1000
3     1000
4     1155
5     1000
6     1000
7    17005
Name: label, dtype: int64

In [ ]:
label7 = OV_train.loc[(OV_train.label == 7)].sample(frac=1).reset_index(drop=True)[:1155]

In [ ]:
other_label = OV_train[OV_train['label'] != 7]

In [ ]:
train = pd.concat([label7, other_label])
train

,review,corrected_reply,store,label
0,항상 잘먹고 있습니다. 맛도 괜찮고 양도많고 배달도 빨라 좋습니다. 근데 오늘은 간...,안녕하세요 우선 만족할 만한 식사를 대접해 드리지 못해 죄송합니다 제가 병원 예약 ...,NaN,7
1,맛있게 잘 먹었습니다,사진 리뷰 감사합니다,NaN,7
2,맛있게 잘먹었습니다,저희 올 데이 파스타를 찾아주셔서 정말 대단히 감사합니다 이렇게 맛있게 드셔주셨다니...,"올데,올데이,올데이파",7
3,여자친구 시켜줬더니 좋아합니다 감사합니다.,안녕하세요! 주문해 주셔서 감사드립니다 디저트와 커피 필승 조합! 다음에 같이 드실...,NaN,7
4,초코 케이크 진짜 너무 맛있어요,리뷰 감사합니다! 디저트 하나하나 맛있게 제공해 드리겠습니다! 또 찾아주세요!,NaN,7
...,...,...,...,...
7150,"이벤트 참여라 별 다섯개는 드립니다만, 회의 신선도가 조금 떨어져보입니다. 포장과 ...",저희 윤희 횟집을 찾아주셔서 감사드립니다 고객님의 정성스러운 리뷰에 감동입니다 회가...,"윤희횟집,윤희,윤희횟",6
7151,후기보고시킨건데.모르겠어요,주문 감사합니다.~ 개인적인 취향도 있어서 고객님 의견 존중합니다~ 저희 김치찌개는...,NaN,6
7152,배달도 양도 맘에 든어요. 찜이라 쫌만 더 끓였으면 하네요.,안녕하세요 신 자매 김치찜 김치찌개를 찾아주셔서 진심으로 감사합니다 배달도 양도 맘...,"신자,신자매",6
7153,야채곱창과 직화곱창입니다,안녕하세요. 삽교 원조 두리 곱창입니다. 저희 음식이 입맛에 맞으셨다니 정말 다행이...,"삽교원조,삽교원",6


## 음식 키워드 컬럼화

In [ ]:
food_list = pd.read_csv(f"{DATA_PATH}food.csv").drop_duplicates(ignore_index = True)["대표식품명"]
food_list

0          케이크
1          치즈빵
2         카스텔라
3          츄러스
4         치아바타
        ...   
753      햄채소볶음
754    무생채(무만)
755     참나물겉절이
756     치커리겉절이
757       짜장소스
Name: 대표식품명, Length: 758, dtype: object

In [ ]:
split_pattern = r'\s[\(/\-]s*'

food_list = (food_list.str.split(split_pattern).str.get(0))
food_list

0          케이크
1          치즈빵
2         카스텔라
3          츄러스
4         치아바타
        ...   
753      햄채소볶음
754    무생채(무만)
755     참나물겉절이
756     치커리겉절이
757       짜장소스
Name: 대표식품명, Length: 758, dtype: object

In [ ]:
food_list = [word for word in food_list if len(word) > 1]

In [ ]:
train['corrected_reply'] = train['corrected_reply'].astype(str)

In [ ]:
train['food'] = train['corrected_reply'].apply(lambda x: next((food for food in food_list if food in x), " "))
train['food']

0           
1           
2         라면
3         커피
4           
        ... 
7150        
7151    김치찌개
7152    김치찌개
7153    막창구이
7154        
Name: food, Length: 8310, dtype: object

In [ ]:
train

,review,corrected_reply,store,label,food
0,항상 잘먹고 있습니다. 맛도 괜찮고 양도많고 배달도 빨라 좋습니다. 근데 오늘은 간...,안녕하세요 우선 만족할 만한 식사를 대접해 드리지 못해 죄송합니다 제가 병원 예약 ...,NaN,7,
1,맛있게 잘 먹었습니다,사진 리뷰 감사합니다,NaN,7,
2,맛있게 잘먹었습니다,저희 올 데이 파스타를 찾아주셔서 정말 대단히 감사합니다 이렇게 맛있게 드셔주셨다니...,"올데,올데이,올데이파",7,라면
3,여자친구 시켜줬더니 좋아합니다 감사합니다.,안녕하세요! 주문해 주셔서 감사드립니다 디저트와 커피 필승 조합! 다음에 같이 드실...,NaN,7,커피
4,초코 케이크 진짜 너무 맛있어요,리뷰 감사합니다! 디저트 하나하나 맛있게 제공해 드리겠습니다! 또 찾아주세요!,NaN,7,
...,...,...,...,...,...
7150,"이벤트 참여라 별 다섯개는 드립니다만, 회의 신선도가 조금 떨어져보입니다. 포장과 ...",저희 윤희 횟집을 찾아주셔서 감사드립니다 고객님의 정성스러운 리뷰에 감동입니다 회가...,"윤희횟집,윤희,윤희횟",6,
7151,후기보고시킨건데.모르겠어요,주문 감사합니다.~ 개인적인 취향도 있어서 고객님 의견 존중합니다~ 저희 김치찌개는...,NaN,6,김치찌개
7152,배달도 양도 맘에 든어요. 찜이라 쫌만 더 끓였으면 하네요.,안녕하세요 신 자매 김치찜 김치찌개를 찾아주셔서 진심으로 감사합니다 배달도 양도 맘...,"신자,신자매",6,김치찌개
7153,야채곱창과 직화곱창입니다,안녕하세요. 삽교 원조 두리 곱창입니다. 저희 음식이 입맛에 맞으셨다니 정말 다행이...,"삽교원조,삽교원",6,막창구이


In [ ]:
train.to_csv(f"{DATA_PATH}0831.csv",index=False) # 저장

# 구글 플레이스토어 데이터 XXX



In [ ]:
ps_df = pd.read_csv(f"{DATA_PATH}playstore_df.csv")
ps_df

,content,score,replyContent
0,쿠폰ㅋㅋㅋㅋㅋㅋ 추가) 다른 분이 언급했듯 배민1 쿠폰과 일반 쿠폰 혼재에 따른 편...,2,"o Dan님,\n\n안녕하세요. 배달의민족입니다.\n혹시 이용에 불편함이나, 만족스..."
1,이벤트도 배달앱 중 가장 다양한것같고 돈도 잘 버는만큼 잘 퍼주는 앱 이었던것,2,"문상화님,\n\n안녕하세요. 배달의민족입니다.\n배달의민족 이용 후 좋다고 말씀해주..."
2,이제 한집배달에만 쿠폰주고 점점 망해가는거 같네요,1,"풍기도리님,\n\n안녕하세요. 배달의민족입니다.\n할인 혜택이 부족하여 아쉬움을 드..."
3,"아니 월간쿠폰북에는 배민1, 배달 이래적혀져있는데 쿠폰을한번더눌러보면 알뜰배달, 배...",1,"롤린이배린이 (롤배그올리는채널)님,\n\n안녕하세요. 배달의민족입니다.\n혜택 부족..."
4,좋아요!!,5,"꼬냑소년님,\n\n안녕하세요. 배달의민족입니다.\n배달의민족 서비스 이용에 만족하신..."
...,...,...,...
103645,통신이 아예 안 되네요 4g는 빵빵 터지는데,1,안녕하세요. 배달의민족 입니다. 네트워크 문제로 불편을 드려 죄송합니다.\n\n배달...
103646,배달의 민족 초반때부터 사용했었는데 예전이 오히려 나은거 같아요 위치설정도 왔다리갔...,1,오랜 시간 보내주신 사랑을 먹고 무럭무럭 성장하고 있는 배달의민족 입니다. \n\n...
103647,가끔 와파떠잇는데 와파잡아야댄다고안들어가저요ㅡㅡ,2,와아파이 잡아야서 얼른 주문해야 되는데 계속 물어봐서 짜증나셨죠. 네트워크 오류가 ...
103648,어쩌라는거야.. 서울에서 안산까지 배달해줌ㅡㅡ?,1,안녕하세요 전국구 배달어플 배달의민족 입니다. 안산에서 서울의 업소정보를 보게 되셨...


In [ ]:
tt = ps_df[ps_df['content'].str.contains('맛집|맛있', na = False)]
tt

,content,score,replyContent
13,아니진짜ㅡㅡ하 겁나 화딱지나서 요기요 쓰던가해야지 찜목록 편집기능을 만들어서 복수삭...,1,"김소현님,\n\n안녕하세요. 배달의민족입니다.\n귀한 시간 내어 소중한 의견 주셔서..."
63,맛있게드세요,4,"캄빤준님,\n\n안녕하세요. 배달의민족입니다.\n고객님의 칭찬에 가게 사장님을 대신..."
109,혹시 가게 메모테그 기능 같은 거 만들어주시면 안되나요? (#여기 로제떡볶이 조금 ...,5,"JH K님,\n\n안녕하세요. 배달의민족입니다.\n고객님의 소중한 의견을 내어주셔서..."
155,리뷰 중단요청하면 무조건 임시조치로 30일 내려가는데 5점 아니면 무조건 명예훼손이...,1,"zer0 rlO님,\n\n안녕하세요. 배달의민족입니다.\n배달의민족 앱 이용 시 불..."
300,맛집정보및신속배달굿,4,"김형순님,\n\n안녕하세요. 배달의민족입니다.\n고객님의 즐거운 식사 시간을 함께하..."
...,...,...,...
103583,배달의 맛집 너무 좋아요 메뉴 가격 다 보이고 맛집 추천도 해 주고 ㅋㅋㅋ 이어플로...,5,안녕하세요 맛과 멋이 있는 배달의민!족! 입니다. 배달의민족은 배달음식점 사장님과 ...
103601,친구랑있을때 뭐먹을지 고민많이될때 리뷰보고 맛있는집잘시켜먹었네요 여기서시켜서후회한...,5,안녕하세요. 돈독한 우정을 응원하는 배달의민족입니다. 고객님이 계신 지역 사장님들께...
103619,낯에는 주변 맛집을 찾을수있고 저녁늦게는 야식집을 손쉽게 찾을수 있고 시간 들일 필...,5,안녕하세요. 대박어플 배달의민족 입니다. 10개의 카테고리로 분류되어 있어 음식점을...
103620,집에서 시켜먹을때면 항상 어플가동 맛있는집만 딱 찾아주니 이리저리 전화안해도 되고 ...,5,안녕하세요. 풀가동 중인 배달의민족 입니다. 급야식엔 배달의민족을 가장 먼저 떠올려...


In [ ]:
tmp = tt[-tt['content'].str.contains('서버|블라인드|중단|업소|어플|앱|요기요|배민|민족|쿠팡|쿠폰|메모|랭킹|찜|이벤트|배달료|배달비|리뷰', na = False)]
tmp

,content,score,replyContent
63,맛있게드세요,4,"캄빤준님,\n\n안녕하세요. 배달의민족입니다.\n고객님의 칭찬에 가게 사장님을 대신..."
300,맛집정보및신속배달굿,4,"김형순님,\n\n안녕하세요. 배달의민족입니다.\n고객님의 즐거운 식사 시간을 함께하..."
535,맛있어요 또 먹고싶어욤!!! 최고!!!!!! 맛에 감동~~ 서비스에 감동~~~,5,"김미미님,\n\n안녕하세요. 배달의민족입니다.\n고객님의 칭찬에 가게 사장님을 대신..."
922,어....요즘 여름이여서 설빙수가..? 아~~~~주 맛있게 와요!!! 주님도 통쾌하실끗!,5,"배정님,\n\n안녕하세요. 배달의민족입니다.\n고객님의 칭찬에 가게 사장님을 대신하..."
975,맛있어요^^,5,"호야님,\n\n안녕하세요. 배달의민족입니다.\n고객님의 칭찬과 소중한 리뷰 감사드립..."
...,...,...,...
103327,좋은음식점 까내리고 나쁜음식점 맛있다고 하는 알바들 정리좀 해주세요,5,안녕하세요. 리뷰천국 배달의민족입니다. 배달의민족에 보내주시는 관심과 격려 감사합니...
103328,저희 세트5번 시켰는데요 일단 사실적인 평을 말씀드리겠습니다 세트는 고추잡채+짜장 ...,4,안녕하세요. 맛있는 고추잡채와 짜장 주문할 수 있는 배달의민족입니다. 배달의민족은 ...
103348,나 너땜에살마니쪘다.하지만 후회는안한다.시키는곳마다 맛있으니까. 쵝오다ㅎㅎ,5,안녕하세요. 배달의민족입니다. 이런 급고백! 고객님 사랑합니다. 살 많이 쪘나요? ...
103378,애기낳고나선 매번집에서모임하는대 복불복으로 메뉴도정할수있고ㅋ짠순이친구들사이에서 전화...,5,안녕하세요. 모임준비도 배달의민족 입니다. 배달의민족은 전국의 배달음식점 정보를 수...


In [ ]:
for i in range(1, 6):
    num_rating = tmp[tmp['score'] == i].shape[0]
    print(f"별점{i}점 개수: {num_rating}, {num_rating / tmp.shape[0] * 100}%")

별점1점 개수: 27, 1.411395713538944%
별점2점 개수: 7, 0.36591740721380034%
별점3점 개수: 43, 2.247778358599059%
별점4점 개수: 246, 12.859383167799269%
별점5점 개수: 1590, 83.11552535284893%


In [ ]:
tmp[tmp['score'] == 1]

,content,score,replyContent
2751,감당도 못할 맛집들 오픈하질말던가.. 계속 오류뜨고 시간낭비..,1,"덕자님,\n\n안녕하세요. 배달의민족입니다.\n이용 시 불편을 드린 것 같아 마음이..."
2790,우리동네맛집으로 배달불가지역을 소개시켜 줘서 시간낭비를 하게 만듦,1,"모래시계님,\n\n안녕하세요. 배달의민족입니다.\n배달의민족 이용 시 만족을 드리지..."
3437,자주 주문했던 가게에서 새벽에 재료발주내려야는데 돈이없다고 몇십만원 빌려달라고 문자...,1,"피아님,\n\n안녕하세요. 배달의민족입니다.\n이용 시 불편을 드린 것 같아 마음이..."
6249,후기보고 시켰다가 광고성 후기였는지 맛없는곳 걸리면 너무 속상하네요 그렇다고 별평을...,1,"윤정환님,\n\n안녕하세요. 배달의민족입니다.\n배달의민족에서 주문한 가게에서 불편..."
7941,주소 대체 어떻게 된겁니까? 중계주공9단지 치니까 상계주공9단지가 먼저 나오네요? ...,1,"이승호님,\n\n안녕하세요. 배달의민족입니다.\n앱 이용 시 불편을 드린 것 같아 ..."
9550,맛있게 잘먹었습니다. 하지만 보쌈이 너무 맛이 없어요 심지어 보쌈에서 테이프? 같은...,1,"소일소먹님,\n\n안녕하세요. 배달의민족입니다.\n저품질의 음식을 받아 속상하셨을 ..."
10090,즐겨찾기를 미쳐 못했던 이름 기억안나던 맛집을 찾을수가 없어요.... 왜죠 ? 항상...,1,"HAJUN jun님,\n\n안녕하세요. 배달의민족입니다.\n앱 이용 시 아쉬움을 드..."
10249,만원 할인을 눌렀는데 왜 할인이 안됐냐고 하 내 만원 내놔 음식 맛있어서 참는다,1,"신라면님,\n\n안녕하세요. 배달의민족입니다.\n고객님의 쿠폰 사용에 어려움이 있으..."
13259,언제 부턴가 별 하나만 보여서 이제 바꼈구나 하고 그렇게 눌렀다 장사하시는 분한테 ...,1,"김덕순님, 안녕하세요. 배달의민족입니다. 배달의민족 앱 이용 시 불편을 드린 것 같..."
17323,이제는 지역맛집정보지로 활용예정,1,안녕하세요 배달의민족입니다.\n최근 어려운 경기 상황으로 사장님들의 고충을 세심히 ...


In [ ]:
tmp['content'][19434]

'덕분에 집 주위에 맛있는집들 발견했으니 이젠 소상공인들 위해서 직접전화주문 하려구요. 무리한 판매수수료 아웃!'

In [ ]:
tmp['replyContent'][19434]

'안녕하세요 배달의민족입니다.\n최근 어려운 경기 상황으로 사장님들의 고충을 세심히 배려하지 못하고 새 요금체계로 많은 분들께 혼란과 부담을 끼쳐드렸습니다. \n이에 사장님들을 비롯한 관계 기관 및 각계에서 많은 의견을 주시어 개선 방안을 고민했고, 그 결과로 새 요금체계를 백지화하게 되었습니다. \n상세한 내용은 빠른 시일 내에 구체적으로 공지드릴 예정입니다\n이번 일을 계기로 저희 앱을 통해, 식당에 주문이 더 늘어나고 라이더 분들은 안정적인 소득을 누리시고 이용자분들께서는 좋은 음식을 원하는 곳에서 드실 수 있도록 최선을 다해 뛰겠습니다.\n다시 한번 불편을 느끼신 모든 분들께 깊이 고개 숙여 사과드립니다.'